# Script for Processing Data

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed
randomState = 123
seed(randomState)

Parameters

In [17]:
irina_rnaseq_filename = "../data/Irina_Budunova/expression_table.tsv"
irina_l1000_filename = "../data/Irina_Budunova/NWU_A_Level3_INF_n378x12328.gctx"
l1000_geneinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_gene_info.txt"

l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

Output Filenames

In [18]:
irina_filtered_l1000_output_filename = "../data/processed/Irina_Budunova/NWU_A_Level3_INF_filtered_n{}x{}.f" # samplesx962
irina_filtered_rnaseq_output_filename = "../data/processed/Irina_Budunova/RNA_seq_filtered_n{}x{}.f" # samplesx962

## Load overlap landmark genes

In [19]:
with open(l1000_landmark_gene_list, "r") as f:
    l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(archs4_all_gene_list, "r") as f:
    archs4_all_gene = [x.strip() for x in f.readlines()]
with open(gtex_l1000_landmark_gene_list, "r") as f:
    gtex_l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(gtex_rnaseq_all_gene_list, "r") as f:
    gtex_rnaseq_all_gene = [x.strip() for x in f.readlines()]
    

overlap_landmark_genes = list(set(l1000_landmark_gene).intersection(archs4_all_gene).intersection(gtex_l1000_landmark_gene).intersection(gtex_rnaseq_all_gene))


## Load Irina's data 

In [22]:
# GTEx L1000 data
print('Loading Irina L1000 data.....')
irina_l1000_data = parse_gctx.parse(irina_l1000_filename,convert_neg_666=True).data_df
gene_info = pd.read_csv(l1000_geneinfo_filename,header = 0, sep = '\t')
# create a probe_id to gene name dictionary 
gene_dict = dict(zip([str(x) for x in gene_info['pr_gene_id']], gene_info['pr_gene_symbol']))

# label rows with gene names 
irina_l1000_data.index = [gene_dict[x] for x in irina_l1000_data.index.values]
irina_l1000_data.loc[overlap_landmark_genes, :]
irina_l1000_data = irina_l1000_data.T

Loading Irina L1000 data.....


In [27]:
irina_l1000_data.loc[irina_l1000_data.index.str.startswith("NWU001_HS"), :]

,DDR1,PAX8,GUCA1A,EPHB3,ESRRA,TRADD,PRPF8,CAPNS1,RPL35,RPL28,...,RABEP2,ZNF783,CX3CL1,NPEPL1,ADAP1,LRCH4,MEX3D,EPS8L1,DCAF15,ACTB
cid,,,,,,,,,,,,,,,,,,,,,
NWU001_HS_XH_X1.A2_B45:A04,8.054899,4.975100,5.444950,7.501100,6.186975,8.518149,14.617550,9.529150,12.429000,13.678750,...,9.248350,8.120750,12.233400,5.721400,5.984300,5.540200,9.385099,5.964150,5.742700,13.610399
NWU001_HS_XH_X1.A2_B45:A05,9.228300,4.886750,4.951850,6.899050,7.783300,10.245700,13.383550,10.527950,13.519300,15.000000,...,9.429649,8.364149,11.941650,6.019250,6.404250,4.753925,9.500850,5.576550,6.148550,15.000000
NWU001_HS_XH_X1.A2_B45:A06,7.623000,4.854000,4.957700,6.876325,6.359300,8.290800,14.344200,8.963375,12.902500,14.290450,...,8.821751,7.682201,11.093800,5.216950,5.288800,4.259300,8.710951,6.211300,5.540950,13.225901
NWU001_HS_XH_X1.A2_B45:A07,8.423002,4.588900,5.255000,6.844950,6.061725,8.599251,14.107950,9.394650,12.904050,13.952900,...,9.531651,8.852100,10.794500,5.713250,5.789650,5.064200,8.621301,5.852400,6.059450,14.226749
NWU001_HS_XH_X1.A2_B45:A08,8.508400,5.256200,5.056400,8.160151,7.395950,9.316901,14.739050,9.849950,13.094050,14.803250,...,10.682775,8.563450,10.453700,6.035900,6.670975,5.687500,8.378450,6.799850,6.830150,13.598400
NWU001_HS_XH_X1.A2_B45:A09,7.544450,4.973850,4.878600,7.679451,6.405250,8.430850,14.071050,9.890550,12.575300,14.038800,...,9.689800,7.862300,10.757851,5.788000,6.154050,4.892000,8.237749,5.467800,6.027950,13.982100
NWU001_HS_XH_X1.A2_B45:A10,8.106700,5.298750,5.188450,7.843150,7.289650,9.021501,13.284849,9.183350,13.040501,14.585600,...,10.126500,7.968850,11.680750,5.869950,5.371000,4.691800,8.464399,7.287400,6.150900,13.478500
NWU001_HS_XH_X1.A2_B45:A11,8.299850,4.986650,4.868700,7.779550,6.393500,8.794451,14.285851,9.970500,12.477150,13.999800,...,9.781400,8.017651,10.761749,5.668050,6.694200,5.162700,8.230400,5.406350,5.916475,14.608600
NWU001_HS_XH_X1.A2_B45:A12,9.424100,5.189475,5.070300,8.320149,7.549450,9.412149,14.990101,10.645950,13.356199,15.000000,...,10.949100,8.422950,12.127900,5.962250,6.615950,5.903650,9.071699,5.892400,6.301900,13.793200


In [24]:
# GTEx RNA-seq data
print('Loading Irina RNA-seq data.....')
irina_rnaseq_data = pd.read_csv(irina_rnaseq_filename, sep="\t")

# with open(gtex_rnaseq_filename, "r") as f:
#     lines = f.readlines()

#     sample_line = lines[2]
#     samples = sample_line.split("\t")
#     sample_index = [i for i, x in enumerate(samples) if x in gtex_l1000_data.index]
#     paired_sample_id = [x for i, x in enumerate(samples) if x in gtex_l1000_data.index]

#     gex_paired_sample = list()
#     gene_names = list()
#     for line in lines[3:]:
#         splited = np.array(line.split("\t"))
#         gene_name = splited[1]

#         if gene_name in overlap_landmark_genes:
#             gene_names.append(gene_name)
#             gex_paired_sample.append(splited[sample_index])

#     gtex_rnaseq_data = pd.DataFrame(gex_paired_sample, columns=paired_sample_id, index=gene_names).T
#     gtex_rnaseq_data.reset_index().to_feather(gtex_filtered_rnaseq_output_filename.format(gtex_rnaseq_data.shape[0], gtex_rnaseq_data.shape[1]))

Loading Irina RNA-seq data.....
            Ensembl_ID GeneSymbol  aaf1_control_f  aaf2_gc_f  aaf3_control_f  \
0      ENSG00000211459    MT-RNR1       18.457936  19.243366       18.466469   
1      ENSG00000210082    MT-RNR2       20.838268  21.402914       21.429066   
2      ENSG00000225972   MTND1P23        7.963775   8.192774        7.072634   
3      ENSG00000211896      IGHG1        5.965911   6.268887        6.868607   
4      ENSG00000211592       IGKC        9.796751   9.364219       11.359939   
...                ...        ...             ...        ...             ...   
21334  ENSG00000139648      KRT71        8.028659   8.419590        2.432085   
21335  ENSG00000204897      KRT25        8.677234   9.164054        2.432085   
21336  ENSG00000159450       TCHH        9.441548   9.570659        4.484038   
21337  ENSG00000135443      KRT85        6.488316   6.200392        2.432085   
21338  ENSG00000205076     LGALS7        7.357721   7.039082        3.750872   

       

In [25]:
irina_rnaseq_data

,Ensembl_ID,GeneSymbol,aaf1_control_f,aaf2_gc_f,aaf3_control_f,aaf4_gc_f,aaf5_control_f,aaf6_gc_f,aaf7_control_m,aaf8_gc_m,...,ccf9_control_m,ccf10_gc_m,ccf11_control_f,ccf12_gc_f,ccf13_control_f,ccf14_gc_f,ccf16_control_f,ccf17_gc_f,ccf18_gc_m,ccf19_control_m
0,ENSG00000211459,MT-RNR1,18.457936,19.243366,18.466469,18.749047,19.235885,18.417699,18.398670,19.678597,...,11.040671,10.761389,10.171134,10.438516,12.210163,9.826061,9.571429,10.163543,10.374474,9.901144
1,ENSG00000210082,MT-RNR2,20.838268,21.402914,21.429066,21.710097,20.893213,20.561893,20.984511,21.685312,...,14.462519,14.278215,13.581400,14.016145,14.894854,13.450872,13.331316,13.994524,13.871352,13.326206
2,ENSG00000225972,MTND1P23,7.963775,8.192774,7.072634,6.722771,7.701601,7.093022,7.725506,8.199215,...,2.932237,2.432085,2.432085,2.432085,2.432085,2.432085,2.929006,2.432085,2.970740,2.432085
3,ENSG00000211896,IGHG1,5.965911,6.268887,6.868607,6.176716,7.516769,8.308454,5.234811,5.305440,...,2.932237,4.419762,2.945139,3.754662,6.917215,6.441541,4.297023,3.466172,4.767083,4.837587
4,ENSG00000211592,IGKC,9.796751,9.364219,11.359939,9.888513,8.235351,8.283954,8.528048,8.903467,...,4.200130,5.054333,5.783915,6.294485,8.006104,7.605840,4.932213,5.797007,6.383887,7.352424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21334,ENSG00000139648,KRT71,8.028659,8.419590,2.432085,2.432085,6.015272,4.144476,2.996790,7.960244,...,8.368119,3.509136,3.834954,2.432085,8.452175,8.996007,3.941478,2.432085,8.461258,8.252726
21335,ENSG00000204897,KRT25,8.677234,9.164054,2.432085,2.432085,6.958961,3.517568,2.996790,9.013898,...,8.365255,3.606804,4.055637,2.432085,8.797021,8.815584,4.072229,3.037525,8.987935,8.414278
21336,ENSG00000159450,TCHH,9.441548,9.570659,4.484038,3.902401,5.535770,4.236855,3.874835,9.127608,...,10.052918,5.162351,4.752553,4.074146,9.952239,9.550439,5.109409,4.240080,10.407995,9.421947
21337,ENSG00000135443,KRT85,6.488316,6.200392,2.432085,2.432085,3.894598,3.325107,2.432085,6.757622,...,8.507126,3.777079,3.153930,2.432085,6.675169,7.286530,3.621354,3.037525,7.292224,7.129839


In [51]:
# filter landmark genes and save
filtered_gtex_l1000_data = gtex_l1000_data.loc[:, overlap_landmark_genes]
filtered_gtex_l1000_data.reset_index().to_feather(gtex_filtered_l1000_output_filename.format(filtered_gtex_l1000_data.shape[0], filtered_gtex_l1000_data.shape[1]))